In [1]:
import mne
mne.set_log_level('error')
import matplotlib
import matplotlib.pyplot as plt
from functions import *
matplotlib.use('TkAgg')

### Loading the data

In [2]:
# data_path = "/Users\sunnivlf\Documents/Data Set Alcohol Detector/Subjects"
data_path = "/Users/sunniva/Documents/Prosjektoppgave/Data Set Alcohol Detector/Subjects"
#data_path = "../Data Set Alcohol Detector/Subjects"
p_id = "/" + "sub-P019" + "/" + "ses-S002" + "/" + "sub-P019_ses-S002_task-Default_run-005"
raw = mne.io.read_raw_fif(data_path + "/" + p_id + "_eeg.fif", preload=True)

Plotting the elctrode placement on the scalp

In [3]:
raw.plot_sensors(show_names=True);

### Filtering the data
Highpass filtering is included to remove drifting of the signal, while lowpass filtering will remove power line noise


In [4]:
low_cut = 0.1 
hi_cut  = 30
raw_filt = raw.copy().filter(low_cut, hi_cut)
raw_filt.compute_psd(fmax=125).plot(picks="data", exclude="bads");
raw_filt.plot(highpass=0.1, lowpass=30, duration=200)

<MNEBrowseFigure size 800x800 with 4 Axes>

Choose one channel to use for this experiment. Fp2 is one of the frontal electrodes. The data and time is extracted from the filtered raw data, as numpy arrays.

In [5]:
picks = mne.pick_channels(ch_names = ['Fp2'], include=['Fp2'])
data, time = raw_filt[picks, :]  

The signal is visulaized using matplotlib.

In [6]:
plt.figure(figsize=(10, 4))
plt.plot(time, data[0])
plt.xlabel('Time (s)')
plt.ylabel('EEG Amplitude')
plt.title('EEG Data for Fp2 Channel')
plt.grid(True)
plt.tight_layout()
plt.show()

In [7]:
sample_rate = raw.info['sfreq']
window_size = 8

s1 = 50
s2 = s1 + window_size

delta1 = int(sample_rate*s1)
delta2 = int(sample_rate*s2)

d = data[0][delta1:delta2]
t = time[delta1:delta2]
num_samples = len(d)

In [8]:
plt.figure(figsize=(10, 4))
plt.plot(t, d)
plt.xlabel('Time (s)')
plt.ylabel('EEG Amplitude')
plt.title('EEG Data for Fp2 Channel')
plt.grid(True)
plt.tight_layout()
plt.show()

In [9]:
(freq, fft_freq) = fft(d, num_samples, sample_rate)

In [10]:
f, dt, ssx = plot_stft(d, sample_rate, 50, 5)

In [26]:
imfs = hht(t, d, plot=False)

In [27]:
#cwtm = wt(t, d, sample_rate, w=3.0)

In [21]:
filt_imfs = imfs[0:len(imfs)-1]

# Set the figure size dynamically based on the number of IMFs
plt.figure(figsize=(10, 2*len(filt_imfs)))

for idx, imf in enumerate(filt_imfs):
    env, inst_freq = ht(t, imf, sample_rate, plot=False)
    freq, fft_res = fft(imf, num_samples, sample_rate, plot=False)
    
    ax = plt.subplot(len(filt_imfs), 1, idx+1)
    plt.plot(freq, fft_res)
    plt.xlim(0, 45)
    
    # If it's not the last subplot, remove the x-axis labels to prevent overlap
    if idx < len(filt_imfs) - 1:
        ax.set_xticklabels([])
    else:
        plt.xlabel('Frequency (Hz)')  # Only add x-axis label to the bottom subplot

    plt.ylabel('Amplitude')

# Adjust the layout to prevent overlapping
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.subplots_adjust(bottom=0.06)
plt.show()


In [ ]:
filt_imfs = imfs[0:len(imfs)-1]

plt.figure(figsize=(10, 4*len(filt_imfs)))
for idx, imf in enumerate(filt_imfs):
    f, t, Sxx = plot_stft(imf, sample_rate=sample_rate, nperseg=100, noverlap=10, plot=False)
    plt.subplot(len(imfs)-1, 1, idx+1)
    plt.pcolormesh(t, f, np.abs(Sxx), shading='gouraud')
    # plt.colorbar(label='Magnitude [dB]')
    # plt.title('STFT Magnitude')
    plt.ylabel('f [Hz]')
    # plt.xlabel('Time [sec]')
    plt.ylim([0, 50])
plt.colorbar(label='Magnitude [dB]')
plt.show()

In [14]:
filt_imfs = imfs[0:len(imfs)-1]

# Determine global min and max values for color scale normalization
vmin = np.inf  # start with infinity
vmax = -np.inf  # start with negative infinity

for imf in filt_imfs:
    f, t, Sxx = plot_stft(imf, sample_rate=sample_rate, nperseg=100, noverlap=10, plot=False)
    vmin = min(vmin, np.abs(Sxx).min())  # update vmin
    vmax = max(vmax, np.abs(Sxx).max())  # update vmax

plt.figure(figsize=(10, 4*len(filt_imfs)))

# Create subplots and apply the same vmin and vmax to each
for idx, imf in enumerate(filt_imfs):
    f, t, Sxx = plot_stft(imf, sample_rate=sample_rate, nperseg=100, noverlap=10, plot=False)
    plt.subplot(len(filt_imfs), 1, idx+1)
    plt.pcolormesh(t, f, np.abs(Sxx), shading='gouraud', vmin=vmin, vmax=vmax)  # Apply the same vmin and vmax
    plt.ylabel('f [Hz]')
    plt.ylim([0, 50])

# After creating all subplots, add a single colorbar to the figure
plt.subplots_adjust(right=0.8)  # Adjust subplot to not overlap with colorbar
cbar_ax = plt.gcf().add_axes([0.85, 0.15, 0.05, 0.7])  # Position for the colorbar
plt.colorbar(cax=cbar_ax, label='Magnitude [dB]')

# Add common xlabel
plt.xlabel('Time [sec]')

plt.show()